In [8]:
import gurobipy as gp
from gurobipy import Model, GRB, quicksum
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np
import seaborn as sns
from IPython.display import display
from matplotlib.ticker import MaxNLocator
import random

import random
from collections import Counter

# Parameters (统一使用)
m = 8
K = 5
L_levels = [25, 20, 15, 12, 10]
E_levels = [13, 28, 38, 45, 120] 
E_PU_max = [5500] * m 
CPU_capacity = [4000] * m
f_v_levels = [30, 40, 60, 80, 100]
P_idle = [16] * m
P_max = [40] * m
T_total = 300
total_tasks = 100  
alpha_1=8000
alpha_2=3
alpha_pu=1.5
n = 100
chain_tasks = {
    0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    1: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
    2: [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
    3: [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
    4: [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
    5: [50, 51, 52, 53, 54, 55, 56, 57],
    6: [58, 59, 60],
    7: [61, 62, 63],
    8: [64, 65, 66],
    9: [67, 68, 69],
    10: [70, 71, 72],
    11: [73, 74, 75],
    12: [76, 77, 78],
    13: [79, 80, 81],
    14: [82, 83, 84],
    15: [85, 86, 87],
    16: [88, 89, 90],
    17: [91, 92, 93],
    18: [94, 95, 96],
    19: [97, 98, 99]
}

num_chains = len(chain_tasks)
buffer = 0.2  
L_max = [len(chain_tasks[k]) * 14 for k in range(num_chains)]


params = (
    alpha_1, alpha_2, alpha_pu,
    E_levels, L_levels, f_v_levels,
    CPU_capacity, P_idle, P_max,
    T_total, chain_tasks, L_max,
    E_PU_max, total_tasks, m
)


In [10]:

def run_greedy_model(params, baseline=False, chain_enabled=None):
    alpha_1, alpha_2, alpha_pu, E_levels, L_levels, f_v_levels, CPU_capacity, \
    P_idle, P_max, T_total, chain_tasks, L_max, E_PU_max, n, m = params

    K = len(E_levels)

    delta = [0] * n
    y = [-1] * n
    w = [-1] * n
    z = [0] * len(chain_tasks)

    pu_remaining_cpu = CPU_capacity.copy()
    pu_workload = [0] * m
    maf_energy = 0
    accepted_chains = 0

    for k, tasks in chain_tasks.items():
        if chain_enabled and chain_enabled.get(k, 1) == 0:
            continue

        task_levels = []
        task_cpus = []
        chain_latency = 0
        feasible = True

        # baseline 模式时，统一使用最高能耗等级 (level=0)
        for i in tasks:
            if baseline:
                selected_level = 0
                level_latency = L_levels[selected_level]
                if chain_latency + level_latency <= L_max[k]:
                    task_levels.append(selected_level)
                    task_cpus.append(f_v_levels[selected_level])
                    chain_latency += level_latency
                else:
                    feasible = False
                    break
            else:
                selected_level = -1
                for level in range(K):
                    if chain_latency + L_levels[level] <= L_max[k]:
                        selected_level = level
                        chain_latency += L_levels[level]
                        break
                if selected_level == -1:
                    feasible = False
                    break
                task_levels.append(selected_level)
                task_cpus.append(f_v_levels[selected_level])

        if not feasible:
            continue

        best_pu = -1
        min_energy_increase = float('inf')

        for j in range(m):
            total_task_cpu = sum(task_cpus)
            total_task_latency = sum(L_levels[lv] for lv in task_levels)

            # 仅检查PU CPU容量约束 (已移除PU_max_latency检查)
            if pu_remaining_cpu[j] >= total_task_cpu:
                predicted_workload = pu_workload[j] + total_task_latency
                idle_time = max(0, T_total - predicted_workload)
                pu_e = P_idle[j] * idle_time + (P_max[j] - P_idle[j]) * predicted_workload
                energy_increase = pu_e - (P_idle[j] * max(0, T_total - pu_workload[j]) + (P_max[j] - P_idle[j]) * pu_workload[j])

                if energy_increase < min_energy_increase and pu_e <= E_PU_max[j]:
                    min_energy_increase = energy_increase
                    best_pu = j

        if best_pu == -1:
            continue

        # 成功分配任务和链激活
        z[k] = 1
        for idx, i in enumerate(tasks):
            delta[i] = 1
            y[i] = task_levels[idx]
            w[i] = best_pu
            pu_remaining_cpu[best_pu] -= task_cpus[idx]
            pu_workload[best_pu] += L_levels[task_levels[idx]]
            maf_energy += E_levels[task_levels[idx]]

        accepted_chains += 1

    # PU 能耗计算
    pu_energy = sum(
        P_idle[j] * max(0, T_total - pu_workload[j]) + (P_max[j] - P_idle[j]) * pu_workload[j]
        for j in range(m)
    )

    total_energy = maf_energy + pu_energy
    acceptance_rate = accepted_chains / len(chain_tasks)
    objective = alpha_1 * accepted_chains - alpha_2 * maf_energy - alpha_pu * pu_energy

    return {
        "delta": delta,
        "w": w,
        "y": y,
        "z": z,
        "objective": objective,
        "total_energy": total_energy,
        "acceptance_rate": acceptance_rate,
        "accepted_chains": accepted_chains,
        "maf_energy": maf_energy,
        "pu_energy": pu_energy
    }



In [10]:
# Baseline模式
baseline_result = run_greedy_model(params, baseline=True)

# Optimal模式
optimal_result = run_greedy_model(params, baseline=False)





In [11]:
print("==== Objective 对比 ====")
print(f"Greedy Baseline Objective: {baseline_result['objective']}")
print(f"Greedy Optimal Objective: {optimal_result['objective']}")


print("\n==== Energy对比 ====")
print(f"Greedy Baseline Energy: {baseline_result['total_energy']}")
print(f"Greedy Optimal Energy: {optimal_result['total_energy']}")

print("\n==== Acceptance Rate对比 ====")
print(f"Greedy Baseline Acceptance: {baseline_result['accepted_chains']}/{len(chain_tasks)}")
print(f"Greedy Optimal Acceptance: {optimal_result['accepted_chains']}/{len(chain_tasks)}")



==== Objective 对比 ====
Greedy Baseline Objective: -57600.0
Greedy Optimal Objective: -57600.0

==== Energy对比 ====
Greedy Baseline Energy: 38400
Greedy Optimal Energy: 38400

==== Acceptance Rate对比 ====
Greedy Baseline Acceptance: 0/20
Greedy Optimal Acceptance: 0/20


In [2]:
def run_heuristic_model_with_diagnostics(alpha_1=2500, alpha_2=10, alpha_3=1, baseline=False):
    import time
    start_time = time.time()

    delta = [0] * n
    y = [-1] * n
    w = [-1] * n
    z = [0] * num_chains

    pu_remaining_cpu = CPU_capacity[:]
    pu_workload = [0] * m
    accepted_chains = 0

    task_energy_levels = [-1] * n
    pu_task_map = {j: [] for j in range(m)}

    print("\n===== [Greedy 链诊断信息] =====")
    for k in range(num_chains):
        task_list = chain_tasks[k]
        task_energy_per_chain = []
        task_cpu_usage = []
        chain_latency = 0
        chain_cpu = 0
        feasible = True

        for i in task_list:
            assigned = False

            if baseline:
                level = K - 1  # 强制最高能耗等级
                latency = L_levels[level]
                cpu = f_v_levels[level]
                task_energy_per_chain.append(level)
                task_cpu_usage.append(cpu)
                chain_latency += latency
                chain_cpu += cpu
                assigned = True
            else:
                for level in reversed(range(K)):  # 从低延迟等级开始
                    latency = L_levels[level]
                    cpu = f_v_levels[level]
                    task_energy_per_chain.append(level)
                    task_cpu_usage.append(cpu)
                    chain_latency += latency
                    chain_cpu += cpu
                    assigned = True
                    break

            if not assigned:
                feasible = False
                break

        total_chain_latency = chain_latency + (len(task_list) - 1)
        if not feasible or total_chain_latency > L_max[k]:
            print(f"❌ 链 {k} 被跳过：延迟约束不满足（总延迟 ≈ {total_chain_latency}, 上限 = {L_max[k]}）")
            continue

        assigned_to_pu = False
        for j in range(m):
            # PU 剩余资源判断
            if pu_remaining_cpu[j] >= chain_cpu:
                # 预测 PU 分配新 workload 后的能耗
                new_workload = pu_workload[j] + sum(L_levels[task_energy_per_chain[idx]] for idx in range(len(task_list)))
                new_pu_energy = P_idle[j] * (T_total - new_workload) + (P_max[j] - P_idle[j]) * new_workload
                if new_pu_energy > E_PU_max[j]:
                    continue  # PU 能耗限制不满足，跳过这个 PU

                # 分配成功
                for idx, i in enumerate(task_list):
                    delta[i] = 1
                    y[i] = task_energy_per_chain[idx]
                    w[i] = j
                    task_energy_levels[i] = task_energy_per_chain[idx]
                    pu_task_map[j].append(i)
                    pu_remaining_cpu[j] -= task_cpu_usage[idx]
                    pu_workload[j] += L_levels[task_energy_per_chain[idx]]
                z[k] = 1
                accepted_chains += 1
                assigned_to_pu = True
                print(f"✅ 链 {k} 被成功激活（PU {j}, 总延迟 = {total_chain_latency}, 总CPU = {chain_cpu}）")
                break

        if not assigned_to_pu:
            print(f"❌ 链 {k} 被跳过：延迟满足但所有 PU 不满足能耗/资源限制（CPU = {chain_cpu}）")

    # === 能耗与目标函数 ===
    maf_energy = sum(E_levels[y[i]] for i in range(n) if y[i] != -1)
    pu_energy = sum(
        P_idle[j] * (T_total - pu_workload[j]) +
        (P_max[j] - P_idle[j]) * pu_workload[j]
        for j in range(m)
    )
    total_energy = maf_energy + pu_energy
    acceptance_rate = accepted_chains / num_chains
    runtime = time.time() - start_time
    objective = alpha_1 * sum(z) - alpha_2 * maf_energy - alpha_3 * pu_energy

    print(f"\n✅ Greedy Summary: 接受 {accepted_chains}/{num_chains} 条链, acceptance_rate = {acceptance_rate:.2%}")

    return {
        "objective": objective,
        "acceptance_rate": acceptance_rate,
        "total_energy": total_energy,
        "runtime": runtime,
        "details": {
            "mode": "Baseline" if baseline else "Non-Baseline",
            "accepted_chains": accepted_chains,
            "total_pu_energy": pu_energy,
            "total_task_energy": maf_energy,
            "activated_chain_list": z,
            "task_energy_levels": task_energy_levels,
            "pu_task_assignment": pu_task_map
        }
    }




In [9]:
result_greedy = run_heuristic_model_with_diagnostics(alpha_1=8000, alpha_2=3, alpha_3=1, baseline=False)



===== [Greedy 链诊断信息] =====
❌ 链 0 被跳过：延迟满足但所有 PU 不满足能耗/资源限制（CPU = 1000）
❌ 链 1 被跳过：延迟满足但所有 PU 不满足能耗/资源限制（CPU = 1000）
❌ 链 2 被跳过：延迟满足但所有 PU 不满足能耗/资源限制（CPU = 1000）
❌ 链 3 被跳过：延迟满足但所有 PU 不满足能耗/资源限制（CPU = 1000）
❌ 链 4 被跳过：延迟满足但所有 PU 不满足能耗/资源限制（CPU = 1000）
✅ 链 5 被成功激活（PU 0, 总延迟 = 87, 总CPU = 800）
✅ 链 6 被成功激活（PU 1, 总延迟 = 32, 总CPU = 300）
✅ 链 7 被成功激活（PU 1, 总延迟 = 32, 总CPU = 300）
✅ 链 8 被成功激活（PU 2, 总延迟 = 32, 总CPU = 300）
✅ 链 9 被成功激活（PU 2, 总延迟 = 32, 总CPU = 300）
✅ 链 10 被成功激活（PU 3, 总延迟 = 32, 总CPU = 300）
✅ 链 11 被成功激活（PU 3, 总延迟 = 32, 总CPU = 300）
✅ 链 12 被成功激活（PU 4, 总延迟 = 32, 总CPU = 300）
✅ 链 13 被成功激活（PU 4, 总延迟 = 32, 总CPU = 300）
✅ 链 14 被成功激活（PU 5, 总延迟 = 32, 总CPU = 300）
✅ 链 15 被成功激活（PU 5, 总延迟 = 32, 总CPU = 300）
✅ 链 16 被成功激活（PU 6, 总延迟 = 32, 总CPU = 300）
✅ 链 17 被成功激活（PU 6, 总延迟 = 32, 总CPU = 300）
✅ 链 18 被成功激活（PU 7, 总延迟 = 32, 总CPU = 300）
✅ 链 19 被成功激活（PU 7, 总延迟 = 32, 总CPU = 300）

✅ Greedy Summary: 接受 15/20 条链, acceptance_rate = 75.00%
